In [37]:
import os
import pandas as pd
import random
#import matplotlib.pyplot as plt

In [59]:
path2 = "/Users/Richard/PycharmProjects/NeuroLab_3.10/WordList/EmoIma/ESM1_categorised.csv"
path1 = "/Users/Richard/PycharmProjects/NeuroLab_3.10/PsychoPy/lists/Emo_categorised.csv"
Emo = pd.read_csv(path1, sep=",")
#Emo.head(50)

In [60]:
Emo

,WORD,IM_Y_M_N,IM_Y_F_N,IM_O_M_N,IM_O_F_N,IM_Y_M_MEAN,IM_Y_F_MEAN,IM_O_M_MEAN,IM_O_F_MEAN,IM_Y_M_SD,...,EM_Y_MEAN,EM_O_MEAN,EM_Y_SD,EM_O_SD,F_Unknown_Y,F_Unknown_O,Kategorie,WORD_length,length_group,Is_plant
0,Aal,11,10,12,10,90.55,93.40,97.33,90.50,12.30,...,5.83,36.16,36.58,41.77,0.0,0.0,nature_based_non_human,3,short_word,NaN
1,Aas,10,10,13,11,73.10,57.50,80.69,59.64,20.44,...,-49.37,-62.74,24.86,32.97,0.0,0.0,NaN,3,short_word,NaN
2,Abend,10,11,10,12,53.80,52.27,59.90,74.08,28.96,...,42.83,50.47,27.44,26.39,0.0,0.0,NaN,5,short_word,NaN
3,Abendbrot,9,11,13,10,64.89,62.64,84.85,82.50,18.72,...,28.79,59.00,23.44,29.92,0.0,0.0,NaN,9,long_word,NaN
4,Abfall,8,10,13,9,73.38,77.80,67.46,60.89,28.84,...,-39.78,-45.63,23.08,28.66,0.0,0.0,NaN,6,short_word,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2587,Zwieback,8,11,12,11,80.00,92.00,95.00,93.27,22.70,...,15.94,42.42,26.83,28.69,0.0,0.0,NaN,8,long_word,NaN
2588,Zwiebel,9,11,12,10,90.67,95.18,93.25,93.20,12.05,...,13.39,9.68,27.29,34.71,0.0,0.0,nature_based_non_human,7,long_word,plant
2589,Zwielicht,9,11,10,12,45.11,30.27,57.60,52.33,28.84,...,-29.72,-30.30,28.49,34.31,0.0,0.0,NaN,9,long_word,NaN
2590,Zwilling,11,10,12,11,70.45,72.70,76.08,83.73,27.48,...,28.89,35.95,29.80,35.93,0.0,0.0,nature_based_human,8,long_word,NaN


## Initial categorisation to filter lists 

In [39]:
#Define Conditions

#set threshold
lower_threshold_emo = -33
upper_threshold_emo = 33

#set threshold Ima
lower_threshold_ima = 33
upper_threshold_ima = 66

# Define emo conditions
emo_mid_condition = (Emo["EM_Y_MEAN"] < upper_threshold_emo) & (Emo["EM_O_MEAN"] < upper_threshold_emo) & (Emo["EM_Y_MEAN"] > lower_threshold_emo) & (Emo["EM_O_MEAN"] > lower_threshold_emo)

# Define ima conditions
ima_high_condition = (Emo["IM_Y_MEAN"] > upper_threshold_ima) & (Emo["IM_O_MEAN"] > upper_threshold_ima)
ima_mid_condition = (Emo["IM_Y_MEAN"] < upper_threshold_ima) & (Emo["IM_O_MEAN"] < upper_threshold_ima) & (Emo["IM_Y_MEAN"] > lower_threshold_ima) & (Emo["IM_O_MEAN"] > lower_threshold_ima)


In [61]:
#Add condition column to emo df
# Initialize new columns with default values
Emo['emo_condition'] = 'None'
Emo['ima_condition'] = 'None'

# Apply conditions and set values
Emo.loc[emo_mid_condition, 'emo_condition'] = 'emo_mid'
Emo.loc[ima_high_condition, 'ima_condition'] = 'ima_high'
Emo.loc[ima_mid_condition, 'ima_condition'] = 'ima_mid'

In [62]:
#Emo
#Test results
#Emo[['WORD', "EM_Y_MEAN", "EM_O_MEAN", "IM_Y_MEAN", "IM_O_MEAN", 'emo_condition', 'ima_condition']].head(50)

,WORD,IM_Y_M_N,IM_Y_F_N,IM_O_M_N,IM_O_F_N,IM_Y_M_MEAN,IM_Y_F_MEAN,IM_O_M_MEAN,IM_O_F_MEAN,IM_Y_M_SD,...,EM_Y_SD,EM_O_SD,F_Unknown_Y,F_Unknown_O,Kategorie,WORD_length,length_group,Is_plant,emo_condition,ima_condition
0,Aal,11,10,12,10,90.55,93.40,97.33,90.50,12.30,...,36.58,41.77,0.0,0.0,nature_based_non_human,3,short_word,NaN,None,ima_high
1,Aas,10,10,13,11,73.10,57.50,80.69,59.64,20.44,...,24.86,32.97,0.0,0.0,NaN,3,short_word,NaN,None,None
2,Abend,10,11,10,12,53.80,52.27,59.90,74.08,28.96,...,27.44,26.39,0.0,0.0,NaN,5,short_word,NaN,None,None
3,Abendbrot,9,11,13,10,64.89,62.64,84.85,82.50,18.72,...,23.44,29.92,0.0,0.0,NaN,9,long_word,NaN,None,None
4,Abfall,8,10,13,9,73.38,77.80,67.46,60.89,28.84,...,23.08,28.66,0.0,0.0,NaN,6,short_word,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2587,Zwieback,8,11,12,11,80.00,92.00,95.00,93.27,22.70,...,26.83,28.69,0.0,0.0,NaN,8,long_word,NaN,None,ima_high
2588,Zwiebel,9,11,12,10,90.67,95.18,93.25,93.20,12.05,...,27.29,34.71,0.0,0.0,nature_based_non_human,7,long_word,plant,emo_mid,ima_high
2589,Zwielicht,9,11,10,12,45.11,30.27,57.60,52.33,28.84,...,28.49,34.31,0.0,0.0,NaN,9,long_word,NaN,emo_mid,ima_mid
2590,Zwilling,11,10,12,11,70.45,72.70,76.08,83.73,27.48,...,29.80,35.93,0.0,0.0,nature_based_human,8,long_word,NaN,None,ima_high


## Utils 

In [42]:
def write_to_excel(file_name, df):
    output_path = "/Users/Richard/PycharmProjects/NeuroLab_3.10/PsychoPy/Output"
    file_path = os.path.join(output_path, file_name)
    df.to_excel(file_path, index=False)
    return f"{file_name} writen to {file_path}"


In [43]:
# Function to generate word pairs with different first letters with checks

#TODO: Add function that uses the gen_word pair function and creates a list for all the conditions that you want to have

#condition_pairs = [("emo_mid", "ima_mid"), ("emo_mid", "ima_high")]

# Function to generate word pairs with different first letters from two different DataFrames
def generate_word_pairs(df1, list_of_condition_pairs, num_pairs_per_condition, seed=42):
    
    # Set seed for reproducibility
    random.seed(seed)
    
    #initalize lists
    word_pairs = []
    used_words = []
    
    for cond_pair in list_of_condition_pairs:
        
        #create dfs based on specified conditions
        emo_words = df1[df1['emo_condition']== cond_pair[0]][['WORD', 'emo_condition']]
        emo_words_tuples = list(zip(emo_words['WORD'], emo_words['emo_condition']))
    
        ima_words = df1[df1['ima_condition']== cond_pair[1]][['WORD', 'ima_condition']]
        ima_words_tuples = list(zip(ima_words['WORD'], ima_words['ima_condition']))
    
        pairs_counter = 0
        attempts_counter = 0
        max_attempts = (len(ima_words_tuples)+len(emo_words_tuples))/2
        
        while pairs_counter < num_pairs_per_condition and attempts_counter < max_attempts:
            attempts_counter += 1
            
            word1 = random.choice(emo_words_tuples)  # Draw randomly from emo_words
            word2 = random.choice(ima_words_tuples)  # Draw randomly from ima_words
            
            # Check if both words are not already used in any word pair
            if (word1[0] not in used_words and 
                word2[0] not in used_words and 
                word1[0][0] != word2[0][0]):
                word_pairs.append((word1, word2))
                used_words.append(word1[0])
                used_words.append(word2[0])
                pairs_counter += 1 
        
        if attempts_counter >= max_attempts:
            print(f"Warning: Maximum attempts reached for condition pair {cond_pair}. Only {pairs_counter} pairs were generated.")
    
    # Unzip the list of tuples into two separate lists
    list1, list2 = zip(*word_pairs)
    
    # Further unzip each of these lists if necessary
    word1, condition1 = zip(*list1)
    word2, condition2 = zip(*list2)

    # Create a DataFrame from these lists
    df = pd.DataFrame({
        'Word1': word1,
        'Emo_condition': condition1,
        'Word2': word2,
        'Ima_condition': condition2
    })     
        
    return df

#list_test = generate_word_pairs(Emo,condition_pairs,18)
#print(generate_word_pairs(emo_high,"WORD", ima_high, "WORD", 10))

#list_test

## Generate Nature List

In [67]:
Emo_short_words_nature

,WORD,IM_Y_M_N,IM_Y_F_N,IM_O_M_N,IM_O_F_N,IM_Y_M_MEAN,IM_Y_F_MEAN,IM_O_M_MEAN,IM_O_F_MEAN,IM_Y_M_SD,...,EM_Y_SD,EM_O_SD,F_Unknown_Y,F_Unknown_O,Kategorie,WORD_length,length_group,Is_plant,emo_condition,ima_condition
0,Aal,11,10,12,10,90.55,93.40,97.33,90.50,12.30,...,36.58,41.77,0.00,0.00,nature_based_non_human,3,short_word,NaN,None,ima_high
20,Adler,10,9,11,12,87.90,90.33,94.73,87.75,11.50,...,30.36,34.99,0.00,0.00,nature_based_non_human,5,short_word,NaN,None,ima_high
22,Affe,10,10,14,8,93.80,94.20,94.64,92.38,6.99,...,35.05,30.47,0.00,0.00,nature_based_non_human,4,short_word,NaN,emo_mid,ima_high
26,Ahorn,12,8,13,10,85.67,70.38,88.85,78.50,12.18,...,31.95,26.68,0.00,0.00,nature_based_non_human,5,short_word,plant,None,ima_high
34,Alge,8,11,13,10,67.00,75.55,76.23,84.70,19.86,...,24.93,36.10,0.00,0.00,nature_based_non_human,4,short_word,plant,emo_mid,ima_high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2531,Zebra,10,11,11,12,88.60,96.45,94.18,91.42,17.15,...,25.53,25.56,0.00,0.00,nature_based_non_human,5,short_word,NaN,None,ima_high
2532,Zecke,10,10,12,11,93.30,80.10,72.00,87.18,7.33,...,31.26,46.86,0.00,0.00,nature_based_non_human,5,short_word,NaN,None,ima_high
2533,Zeder,10,10,12,11,57.40,55.80,78.58,67.73,33.29,...,22.80,28.46,0.00,0.03,nature_based_non_human,5,short_word,plant,emo_mid,None
2551,Ziege,13,8,11,12,91.85,91.88,96.82,90.00,9.93,...,23.75,34.36,0.00,0.00,nature_based_non_human,5,short_word,NaN,emo_mid,ima_high


In [63]:
# Filter list to get only nature Words
Emo_words_nature = Emo[Emo['Kategorie'] == 'nature_based_non_human']
Emo_short_words_nature = Emo[(Emo['length_group'] == 'short_word') & (Emo['Kategorie'] == 'nature_based_non_human')]
Emo_long_words_nature = Emo[(Emo['length_group'] == 'long_word') & (Emo['Kategorie'] == 'nature_based_non_human')]

In [85]:
#filter for short plant words
Emo_cat_short_plants = Emo_short_words_nature[(Emo_short_words_nature["Is_plant"] == "plant")]
#filter for short non plant words i.e. animals 
Emo_cat_short_animals = Emo_short_words_nature[(Emo_short_words_nature["Is_plant"].isna())]

#filter for long plant words 
Emo_cat_long_plants = Emo_long_words_nature[(Emo_long_words_nature["Is_plant"] == "plant")]
#filter for short non plant words i.e. animals 
Emo_cat_long_animals = Emo_long_words_nature[(Emo_long_words_nature["Is_plant"].isna())]

## Short Words Nature

In [119]:
#Set Condition Pair to be generated
condition_pairs = [("emo_mid", "ima_high")]
#Generate wordpairs for short words plant + plant
cond_short_nature_plant_plant = generate_word_pairs(Emo_cat_short_plants,condition_pairs,5, -12)
cond_short_nature_plant_plant

,Word1,Emo_condition,Word2,Ima_condition
0,Kümmel,emo_mid,Gras,ima_high
1,Raute,emo_mid,Lilie,ima_high
2,Erbse,emo_mid,Klee,ima_high
3,Alge,emo_mid,Kiefer,ima_high
4,Quitte,emo_mid,Senf,ima_high


In [81]:
#Generate wordpairs for short words animal + animal
cond_short_nature_plant_animal = generate_word_pairs(Emo_cat_short_animals,condition_pairs,5, -21)
cond_short_nature_plant_animal

,Word1,Emo_condition,Word2,Ima_condition
0,Fuchs,emo_mid,Raupe,ima_high
1,Spinne,emo_mid,Kuh,ima_high
2,Vieh,emo_mid,Hummer,ima_high
3,Tölpel,emo_mid,Vogel,ima_high
4,Geier,emo_mid,Tiger,ima_high


In [82]:
#Handpick to mix words form both categories --> animal + plant and plant + animal
pick_plant = generate_word_pairs(Emo_cat_short_plants,condition_pairs,10, 66)
pick_plant

,Word1,Emo_condition,Word2,Ima_condition
0,Busch,emo_mid,Hafer,ima_high
1,Kraut,emo_mid,Weide,ima_high
2,Kresse,emo_mid,Gurke,ima_high
3,Hanf,emo_mid,Wurzel,ima_high
4,Linse,emo_mid,Birke,ima_high
5,Schilf,emo_mid,Tabak,ima_high
6,Erbse,emo_mid,Tulpe,ima_high
7,Spinat,emo_mid,Raute,ima_high
8,Klette,emo_mid,Reis,ima_high
9,Garbe,emo_mid,Korn,ima_high


In [120]:
#Handpick to mix words form both categories --> animal + plant and plant + animal
pick_animal = generate_word_pairs(Emo_cat_short_animals,condition_pairs,10, 80)
pick_animal

,Word1,Emo_condition,Word2,Ima_condition
0,Krabbe,emo_mid,Pudel,ima_high
1,Spinne,emo_mid,Ochse,ima_high
2,Person,emo_mid,Wirt,ima_high
3,Reptil,emo_mid,Otter,ima_high
4,Auster,emo_mid,Zecke,ima_high
5,Rabe,emo_mid,Hund,ima_high
6,Blei,emo_mid,Pony,ima_high
7,Hummer,emo_mid,Gemse,ima_high
8,Ziege,emo_mid,Ameise,ima_high
9,Ferkel,emo_mid,Sirene,ima_high


## Long words Nature

In [87]:
#Generate wordpairs for long words plant + plant
cond_long_nature_plant_plant = generate_word_pairs(Emo_cat_long_plants,condition_pairs,5, -15)
cond_long_nature_plant_plant

,Word1,Emo_condition,Word2,Ima_condition
0,Knoblauch,emo_mid,Apfelbaum,ima_high
1,Strauch,emo_mid,Blumenkohl,ima_high
2,Getreide,emo_mid,Brombeere,ima_high
3,Zwiebel,emo_mid,Karotte,ima_high
4,Lorbeer,emo_mid,Spargel,ima_high


In [97]:
#Generate wordpairs for long words animal + animal
cond_long_nature_animal_animal = generate_word_pairs(Emo_cat_long_animals,condition_pairs,5, 68
                                                    )
cond_long_nature_animal_animal

,Word1,Emo_condition,Word2,Ima_condition
0,Seeigel,emo_mid,Tarantel,ima_high
1,Faultier,emo_mid,Makrele,ima_high
2,Leopard,emo_mid,Seestern,ima_high
3,Jagdhund,emo_mid,Pflanze,ima_high
4,Maulwurf,emo_mid,Waschbär,ima_high


In [107]:
#Handpick to mix words form both categories --> animal + plant and plant + animal
pick_plant_long = generate_word_pairs(Emo_cat_long_plants,condition_pairs,6, 62)
pick_plant_long

,Word1,Emo_condition,Word2,Ima_condition
0,Strauch,emo_mid,Kirsche,ima_high
1,Getreide,emo_mid,Lorbeer,ima_high
2,Unkraut,emo_mid,Radieschen,ima_high
3,Knoblauch,emo_mid,Veilchen,ima_high
4,Zwiebel,emo_mid,Haselnuss,ima_high
5,Papyrus,emo_mid,Wacholder,ima_high


In [112]:
#Handpick to mix words form both categories --> animal + plant and plant + animal
pick_animal = generate_word_pairs(Emo_cat_long_animals,condition_pairs,5, 2)
pick_animal

,Word1,Emo_condition,Word2,Ima_condition
0,Ziegenbock,emo_mid,Flamingo,ima_high
1,Eidechse,emo_mid,Gazelle,ima_high
2,Sprosse,emo_mid,Panther,ima_high
3,Jagdhund,emo_mid,Languste,ima_high
4,Schenkel,emo_mid,Kaninchen,ima_high


In [48]:
cond_new = [("emo_mid", "ima_high")]
generate_word_pairs(Emo_long_words_nature,cond_new,1, 70)
#nature_words_new = pd.concat([cond_short_nature, cond_new, nature_dummy], ignore_index=True)
#nature_words_new

,Word1,Emo_condition,Word2,Ima_condition
0,Habicht,emo_mid,Kolibri,ima_high


## Primacy / Recency Nature

In [121]:
#Condition pair none so it's not a word within the emo/ ima categories
condition_pairs_primacy_recency = [("None", "None")]

#create dummy words and split in 2 dataframes
dummy_words = generate_word_pairs(Emo_words_nature, condition_pairs_primacy_recency, 6, 4)
dummy_words
#primacy_words = dummy_words.iloc[:3]
#recency_words = dummy_words.iloc[3:]

,Word1,Emo_condition,Word2,Ima_condition
0,Haustier,None,Kardinal,None
1,Drossel,None,Papyrus,None
2,Natter,None,Garbe,None
3,Zebra,None,Dahlie,None
4,Hahn,None,Vanille,None
5,Pinie,None,Kraut,None


In [49]:

#Concat all individual dfs
nature_words = pd.concat([primacy_words,cond_short_nature, cond_long_nature, recency_words], ignore_index=True)
nature_words

,Word1,Emo_condition,Word2,Ima_condition
0,Diebstahl,None,Misserfolg,None
1,Gewalt,None,Krieg,None
2,Kompott,None,Vorschlag,None
3,Spore,emo_mid,Bulle,ima_high
4,Araber,emo_mid,Vampir,ima_high
5,Hafer,emo_mid,Gras,ima_high
6,Gaul,emo_mid,Ei,ima_high
7,Wolf,emo_mid,Buche,ima_high
8,Quitte,emo_mid,Bock,ima_high
9,Rohr,emo_mid,Larve,ima_high


In [52]:
nature_words["WORD_length1"] = nature_words["Word1"].apply(len)
nature_words["WORD_length2"] = nature_words["Word2"].apply(len)

mean_length = Emo["WORD_length"].mean()

def length_categorization(word):
    if len(word) < mean_length:
        return "short_word"
    elif len(word) > mean_length:
        return "long_word"
    
nature_words["length_group_emo"] = nature_words["Word1"].apply(length_categorization)
nature_words["length_group_ima"] = nature_words["Word2"].apply(length_categorization)


#Reorganize columns
cols = nature_words.columns.tolist()
cols = ['Word1', 'WORD_length1', 'Emo_condition','length_group_emo','Word2',"WORD_length2",'Ima_condition', "length_group_ima"]


nature_words= nature_words[cols]

#final result
nature_words

,Word1,WORD_length1,Emo_condition,length_group_emo,Word2,WORD_length2,Ima_condition,length_group_ima
0,Diebstahl,9,None,long_word,Misserfolg,10,None,long_word
1,Gewalt,6,None,short_word,Krieg,5,None,short_word
2,Kompott,7,None,long_word,Vorschlag,9,None,long_word
3,Spore,5,emo_mid,short_word,Bulle,5,ima_high,short_word
4,Araber,6,emo_mid,short_word,Vampir,6,ima_high,short_word
5,Hafer,5,emo_mid,short_word,Gras,4,ima_high,short_word
6,Gaul,4,emo_mid,short_word,Ei,2,ima_high,short_word
7,Wolf,4,emo_mid,short_word,Buche,5,ima_high,short_word
8,Quitte,6,emo_mid,short_word,Bock,4,ima_high,short_word
9,Rohr,4,emo_mid,short_word,Larve,5,ima_high,short_word


In [41]:
write_to_excel('nature_words_new.xlsx',nature_words_new)

'nature_words_new.xlsx writen to /Users/Richard/PycharmProjects/NeuroLab_3.10/PsychoPy/Output/nature_words_new.xlsx'

## Generate list for non-nature words

In [22]:
Emo.head()

,WORD,IM_Y_M_N,IM_Y_F_N,IM_O_M_N,IM_O_F_N,IM_Y_M_MEAN,IM_Y_F_MEAN,IM_O_M_MEAN,IM_O_F_MEAN,IM_Y_M_SD,...,EM_O_MEAN,EM_Y_SD,EM_O_SD,F_Unknown_Y,F_Unknown_O,Kategorie,WORD_length,length_group,emo_condition,ima_condition
0,Aal,11,10,12,10,90.55,93.40,97.33,90.50,12.30,...,36.16,36.58,41.77,0.0,0.0,nature_based_non_human,3,short_word,None,ima_high
1,Aas,10,10,13,11,73.10,57.50,80.69,59.64,20.44,...,-62.74,24.86,32.97,0.0,0.0,NaN,3,short_word,None,None
2,Abend,10,11,10,12,53.80,52.27,59.90,74.08,28.96,...,50.47,27.44,26.39,0.0,0.0,NaN,5,short_word,None,None
3,Abendbrot,9,11,13,10,64.89,62.64,84.85,82.50,18.72,...,59.00,23.44,29.92,0.0,0.0,NaN,9,long_word,None,None
4,Abfall,8,10,13,9,73.38,77.80,67.46,60.89,28.84,...,-45.63,23.08,28.66,0.0,0.0,NaN,6,short_word,None,None


In [9]:
# Filter list to get only nature Words
Emo_words_non_nature = Emo[Emo['Kategorie'].isna()]
Emo_short_words_non_nature = Emo_words_non_nature[(Emo_words_non_nature['length_group'] == 'short_word')]
Emo_long_words_non_nature = Emo_words_non_nature[(Emo_words_non_nature['length_group'] == 'long_word')]

In [55]:
#Generate non-nature words for recency
cond_non_nature_recency = generate_word_pairs(Emo_words_non_nature, condition_pairs_primacy_recency, 3, -14)

cond_non_nature_recency

,Word1,Emo_condition,Word2,Ima_condition
0,Bunker,None,Zuwachs,None
1,Gesicht,None,Makel,None
2,Weihnacht,None,Leistung,None


In [14]:
cond_non_new = [("emo_mid", "ima_high")]
non_nature_short = generate_word_pairs(Emo_short_words_non_nature,cond_non_new,3, -7000)
non_nature_short["WORD_length1"] = non_nature_short["Word1"].apply(len)
non_nature_short["WORD_length2"] = non_nature_short["Word2"].apply(len)

non_nature_short

,Word1,Emo_condition,Word2,Ima_condition,WORD_length1,WORD_length2
0,Lokal,emo_mid,Beton,ima_high,5,5
1,Mast,emo_mid,Stall,ima_high,4,5
2,Kabel,emo_mid,Murmel,ima_high,5,6


In [17]:
cond_non_new = [("emo_mid", "ima_high")]
non_nature_long = generate_word_pairs(Emo_long_words_non_nature,cond_non_new,10, -7000)
non_nature_long["WORD_length1"] = non_nature_long["Word1"].apply(len)
non_nature_long["WORD_length2"] = non_nature_long["Word2"].apply(len)
non_nature_long

,Word1,Emo_condition,Word2,Ima_condition,WORD_length1,WORD_length2
0,Heldenmut,emo_mid,Bauernhof,ima_high,9,9
1,Institut,emo_mid,Schinken,ima_high,8,8
2,Ergebnis,emo_mid,Mikrofon,ima_high,8,8
3,Pfeffer,emo_mid,Biskuit,ima_high,7,7
4,Toilette,emo_mid,Schachtel,ima_high,8,9
5,Billard,emo_mid,Vitrine,ima_high,7,7
6,Purzelbaum,emo_mid,Karussell,ima_high,10,9
7,Maschine,emo_mid,Backofen,ima_high,8,8
8,Ausgang,emo_mid,Brosche,ima_high,7,7
9,Einzelheit,emo_mid,Lexikon,ima_high,10,7


In [ ]:
condition_pairs_primacy_recency = [("emo_mid", "ima_high")]
non_nature_new = generate_word_pairs(Emo,condition_pairs_primacy_recency,2, -14)
non_nature_new

In [28]:

#Set Condition Pair to be generated
condition_pairs = [("emo_mid", "ima_high")]

#Generate wordpairs for short words and for long words
cond_short_non_nature = generate_word_pairs(Emo_short_words_non_nature, condition_pairs, 18)
cond_long_non_nature = generate_word_pairs(Emo_long_words_non_nature, condition_pairs, 18)

#Concat all individual dfs (use same primacy/ recency words as in nature word list)
non_nature_words = pd.concat([primacy_words, cond_short_non_nature, cond_long_non_nature, recency_words], ignore_index=True)

non_nature_words["WORD_length1"] = non_nature_words["Word1"].apply(len)
non_nature_words["WORD_length2"] = non_nature_words["Word2"].apply(len)

mean_length = Emo["WORD_length"].mean()

#apply length categorization
non_nature_words["length_group_emo"] = non_nature_words["Word1"].apply(length_categorization)
non_nature_words["length_group_ima"] = non_nature_words["Word2"].apply(length_categorization)

#Reorganize columns
cols = non_nature_words.columns.tolist()
cols = ['Word1', 'WORD_length1', 'Emo_condition', 'length_group_emo', 'Word2', "WORD_length2", 'Ima_condition',
        "length_group_ima"]

non_nature_words = non_nature_words[cols]

#final result
non_nature_words

,Word1,WORD_length1,Emo_condition,length_group_emo,Word2,WORD_length2,Ima_condition,length_group_ima
0,Diebstahl,9,None,long_word,Misserfolg,10,None,long_word
1,Gewalt,6,None,short_word,Krieg,5,None,short_word
2,Kompott,7,None,long_word,Vorschlag,9,None,long_word
3,Wodka,5,emo_mid,short_word,Dreck,5,ima_high,short_word
4,Bagger,6,emo_mid,short_word,Kappe,5,ima_high,short_word
5,Kelch,5,emo_mid,short_word,Haus,4,ima_high,short_word
6,Gang,4,emo_mid,short_word,Deckel,6,ima_high,short_word
7,Stufe,5,emo_mid,short_word,Büste,5,ima_high,short_word
8,Tuch,4,emo_mid,short_word,Napf,4,ima_high,short_word
9,Eis,3,emo_mid,short_word,Handy,5,ima_high,short_word


In [31]:
write_to_excel('non_nature_words.xlsx',non_nature_words)

'non_nature_words.xlsx writen to /Users/Richard/PycharmProjects/NeuroLab_3.10/PsychoPy/Output/non_nature_words.xlsx'

In [2]:
def write_to_excel(file_name, df):
    output_path = "/Users/Richard/PycharmProjects/NeuroLab_3.10/PsychoPy/Output"
    file_path = os.path.join(output_path, file_name)
    df.to_excel(file_path, index=False)
    return f"{file_name} writen to {file_path}"


## Split lists equally with short and long words alike

In [34]:
    output_path = "/Users/Richard/PycharmProjects/NeuroLab_3.10/PsychoPy/lists"
    file_path = os.path.join(output_path, "non_nature_words_final.xlsx")
    non_nature_words_final.to_excel(file_path, index=False)

In [28]:
# Shuffle the DataFrame rows
non_nature_words_final = non_nature_words_final.sample(frac=1).reset_index(drop=True)

## Some checks to test if there are errors in the list

In [123]:
nautre_words_final = pd.read_excel("/Users/Richard/PycharmProjects/NeuroLab_3.10/PsychoPy/lists/nature/new_nature_words_final.xlsx")

nautre_words_final

,Word1,WORD_length1,Emo_condition,length_group_emo,Word2,WORD_length2,Ima_condition,length_group_ima,Randomize,Combination
0,Biene,5,emo_mid,short_word,Dorsch,6,ima_high,short_word,False,Primacy
1,Buche,5,emo_mid,short_word,Zobel,5,ima_high,short_word,False,Primacy
2,Flieder,7,emo_mid,short_word,Kardinal,8,ima_high,short_word,False,Primacy
3,Efeu,4,emo_mid,short_word,Moos,4,ima_high,short_word,True,Plant / Plant
4,Quitte,6,emo_mid,short_word,Lilie,5,ima_high,short_word,True,Plant / Plant
5,Zeder,5,emo_mid,short_word,Farn,4,ima_high,short_word,True,Plant / Plant
6,Hanf,4,emo_mid,short_word,Roggen,6,ima_high,short_word,True,Plant / Plant
7,Alge,4,emo_mid,short_word,Bambus,6,ima_high,short_word,True,Plant / Plant
8,Fuchs,5,emo_mid,short_word,Raupe,5,ima_high,short_word,True,Animal / Animal
9,Spinne,6,emo_mid,short_word,Kuh,3,ima_high,short_word,True,Animal / Animal


In [124]:
#Check to see if a word from Emo Word list is also in Ima Word list
nautre_words_final["Word1"].isin(nautre_words_final["Word2"])

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
Name: Word1, dtype: bool

In [40]:
#check if a word appears more then once

# Count the occurrences of each word in the 'words' column
word_counts_emo = nature_words_new['Word1'].value_counts()
word_counts_ima = nature_words_new['Word2'].value_counts()

# Filter words that appear more than once
repeated_words_emo = word_counts_emo[word_counts_emo > 1]
repeated_words_ima = word_counts_ima[word_counts_ima > 1]

print("Words that appear more than once:")
print(repeated_words_emo)

print("Words that appear more than once:")
print(repeated_words_ima)

Words that appear more than once:
Series([], Name: count, dtype: int64)
Words that appear more than once:
Series([], Name: count, dtype: int64)
